In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import pandas as pd
import numpy as np
import h5py
import joblib

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
import warnings

# 모든 FutureWarning 무시
warnings.simplefilter(action='ignore', category=FutureWarning)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

2025-05-06 15:18:25.816410: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746544705.826994   20327 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746544705.830246   20327 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


- 새로 접하는 slide의 Target 값을 몇몇 

# Data Processing

In [2]:
def load_data(filename):
    """
    이미지를 불러옵니다.
    Parameters:
        filename: str
            h5 파일에서 데이터를 불러옵니다.
    Returns:
        np.ndarray, pd.DataFrame, np.ndarray, 
        train 이미지, train spot 정보, test 이미지, test spot 정보
    """
    images, spots = list(), list()
    with h5py.File(filename, "r") as h5file:
        train_images = h5file["images/Train"]
        train_spots = h5file["spots/Train"]
    
        num_train_slides = len(train_images)
        # Train 이미지를 불러옵니다.
        # 하나의 텐서로 만들기 위해 이미지의 크기를 2000x2000으로 균일하게 만듭니다.
        for i, slide_name in enumerate(train_images.keys()):
            image = (np.array(train_images[slide_name]) * 255).astype('uint8')
            p1 = 2000 - image.shape[0]
            p2 = 2000 - image.shape[1]
            images.append(
                np.pad(image, [(0, p1), (0, p2), (0, 0)], 'edge')
            )
            spots.append(pd.DataFrame(np.array(train_spots[slide_name])).assign(slide = i))
            if slide_name == 'S_2':
                spots[-1] = spots[-1].assign(
                    x = lambda x: x['x'] - 60,
                    y = lambda x: x['y'] - 60,
                )
    # EfficientNet의 형식으로 바꿉니다.
    with tf.device('/CPU:0'):
        images = tf.constant(tf.keras.applications.efficientnet.preprocess_input(images))
    df_spots = pd.concat(spots).reset_index(drop = True)
    return images, df_spots

def make_img_proc_info(df, img_with, img_height):
    return df.assign(
        left = lambda x: x['x'] - img_width // 2,
        right = lambda x: x['x'] + img_width // 2,
        top = lambda x: x['y'] - img_height // 2,
        bottom = lambda x: x['y'] + img_height // 2,
        lpad = lambda x: -(x['left'].where(x['left'] < 0, 0)),
        rpad = lambda x: -(2000 - x['right']).where(x['right'] > 2000, 0),
        tpad = lambda x: -(x['top'].where(x['top'] < 0, 0)),
        bpad = lambda x: -(2000 - x['bottom']).where(x['bottom'] > 2000, 0)
    ).assign(
        left = lambda x: x['left'].clip(0, 2000),
        right = lambda x: x['right'].clip(0, 2000),
        top = lambda x: x['top'].clip(0, 2000),
        bottom = lambda x: x['bottom'].clip(0, 2000),
    )

def create_tf_ds(df):
    if target in df.columns:
        return tf.data.Dataset.from_tensor_slices(
            ({
                i: df[i] for i in ['left', 'right', 'top', 'bottom', 'slide', 'lpad', 'rpad', 'tpad', 'bpad']
            }, (df['C1'] > df['C2']).astype('int'))
        )
    else:
        return tf.data.Dataset.from_tensor_slices({
            i: df[i] for i in ['left', 'right', 'top', 'bottom', 'slide', 'lpad', 'rpad', 'tpad', 'bpad']
        })

def proc_images(X, images):
    return tf.cast(tf.pad(
        images[X['slide'], X['left']:X['right'], X['top']:X['bottom'], :], 
        paddings = [(X['lpad'], X['rpad']), (X['tpad'], X['bpad']), (0, 0)],
        constant_values=1
    ), tf.float32) / 255.0

augmentation_layers = [
    #tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.5),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1),
]

def data_augmentation(x):
    for layer in augmentation_layers:
        x = layer(x)
    return x

images, df_spots = load_data("data/elucidata_ai_challenge_data.h5")
target = 'slide'

I0000 00:00:1746544712.913878   20327 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5520 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(14, 7))
ax = ax.flatten()
for i, slide_name in enumerate(range(7)):
    image = images[i]
    x, y = df_spots.loc[df_spots['slide'] == i, "x"], df_spots.loc[df_spots['slide'] == i, "y"]

    ax[i].imshow(image, aspect="auto")
    ax[i].scatter(x, y, color="red", s=1, alpha=0.4)  # Overlay spot locations
    ax[i].set_title(slide_name)
    ax[i].axis('off')

plt.tight_layout()
plt.show()

In [4]:
def create_model(img_width, img_height):
    input_shape = (img_width, img_height, 3)
    inputs = tf.keras.Input(shape = input_shape)
    enet_, _ = joblib.load('model/ae_encoder_b0_obj.joblib')
    enet = tf.keras.applications.EfficientNetB0(
        include_top = False, 
        #weights = 'imagenet',
        input_shape = input_shape,
        pooling = 'avg'
    )
    enet.set_weights(enet_)
    enet.trainable = False
    """
    for i in enet.layers:
        if not isinstance(i, tf.keras.layers.BatchNormalization):
            i.trainable = True
    """
    x = enet(inputs)
    #x = tf.keras.layers.Dropout(0.5)(x)
    d1 = tf.keras.layers.Dense(
        256, activation = 'swish', kernel_initializer = 'HeUniform'
    )
    x = d1(x)
    d2 = tf.keras.layers.Dense(1)
    outputs = d2(x)
    m = tf.keras.models.Model(inputs, outputs)
    return m, (enet, d1, d2)

In [5]:
from sklearn.model_selection import GroupKFold, StratifiedShuffleSplit
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
img_width = 224
img_height = 224
df_spots = make_img_proc_info(df_spots, img_width, img_height)

gkf = GroupKFold(n_splits = 6)
scores = list()
oofs = list()
for i, (train_idx, valid_idx) in enumerate(
    gkf.split(df_spots[['x', 'y']], df_spots['slide'], groups = df_spots['slide'])
):

    m, layers = create_model(img_width, img_height)
    learning_rate = 1e-5
    #target_proc = FunctionTransformer(lambda x: x, lambda x: x)
    batch_size = 32
    epochs = 10
    step = ''
    ds_cv_train = create_tf_ds(
        df_spots.iloc[train_idx].pipe(
            lambda x: pd.concat([
                x, x.sample(n = batch_size - (len(x) % batch_size))
            ])
        )
    ).shuffle(5000).map(
        lambda X, y: (proc_images(X, images), y)
    ).map(
        lambda X, y: (data_augmentation(X), y)
    ).batch(batch_size)

    if valid_idx is not None:
        ds_valid = create_tf_ds(df_spots.iloc[valid_idx]).map(
            lambda X, y: (proc_images(X, images), y)
        ).batch(batch_size)
    else:
        ds_valid = None
    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=learning_rate,
        decay_steps=3000,
        alpha=0.1
    )
    #    lr_schedule = learning_rate
    m.compile(
        loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule),
        metrics = [tf.keras.metrics.BinaryAccuracy()]
    )

    hist = m.fit(ds_cv_train, epochs = epochs, validation_data = ds_valid)

Epoch 1/10


I0000 00:00:1746544786.920506   20376 service.cc:148] XLA service 0x7f4b4c14d100 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746544786.920535   20376 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
I0000 00:00:1746544787.772943   20376 cuda_dnn.cc:529] Loaded cuDNN version 90300


  2/190 ━━━━━━━━━━━━━━━━━━━━ 14s 79ms/step - binary_accuracy: 0.7031 - loss: 4.2499 

I0000 00:00:1746544794.714978   20376 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


190/190 ━━━━━━━━━━━━━━━━━━━━ 66s 261ms/step - binary_accuracy: 0.6124 - loss: 1.5775 - val_binary_accuracy: 0.1035 - val_loss: 0.9706
Epoch 2/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 40s 212ms/step - binary_accuracy: 0.6610 - loss: 0.6759 - val_binary_accuracy: 0.0634 - val_loss: 1.4110
Epoch 3/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 40s 211ms/step - binary_accuracy: 0.6751 - loss: 0.6523 - val_binary_accuracy: 0.0925 - val_loss: 1.0670
Epoch 4/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 35s 186ms/step - binary_accuracy: 0.6659 - loss: 0.6468 - val_binary_accuracy: 0.0784 - val_loss: 1.2072
Epoch 5/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 40s 210ms/step - binary_accuracy: 0.6609 - loss: 0.6492 - val_binary_accuracy: 0.0965 - val_loss: 1.0358
Epoch 6/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 37s 194ms/step - binary_accuracy: 0.6665 - loss: 0.6493 - val_binary_accuracy: 0.0819 - val_loss: 1.2288
Epoch 7/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 39s 206ms/step - binary_accuracy: 0.6662 - loss: 0.6407 - val_binary_accuracy: 0.0938 - val_loss: 1.0

KeyboardInterrupt: 